**IMPORTING LIBRARIES AND INITIALIZING DIRECTORIES**

In [2]:
# Import Keras libraries and packages
from keras.applications.resnet import ResNet50
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2

train_dir = "../input/american-sign-language-dataset/dataset/train" 
test_dir = "../input/american-sign-language-dataset/dataset/test"

In [3]:
# Display first image from every class
fig = plt.figure(figsize=(14, 14))
uniq_labels = sorted((f for f in os.listdir(train_dir) if not f.startswith(".")))
for idx, label in enumerate(uniq_labels):
    filepath = train_dir + "/" + label + "/" + label + "2.jpg"
    image = cv2.imread(filepath)
    image = cv2.resize(cv2.imread(filepath), (128, 128))
    ax = fig.add_subplot(5, 8, idx+1, xticks=[], yticks=[])
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig('dataset.png')
    plt.imshow(image)
    ax.set_title('Class: %s'%label)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, 
                               validation_split=0.2) 
train = train_gen.flow_from_directory(train_dir, target_size=(128, 128), subset="training", batch_size=32, class_mode='categorical')
val = train_gen.flow_from_directory(train_dir, target_size=(128, 128), subset="validation", batch_size=32, class_mode='categorical')
test_gen = ImageDataGenerator(rescale=1./255)
test = test_gen.flow_from_directory(test_dir, target_size=(128, 128), class_mode='categorical')

# **RESNET50 MODEL**

In [5]:
classifier_resnet = ResNet50(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

In [6]:
#Not training existing weights for resnet50
for layer in classifier_resnet.layers:
    layer.trainable = False

In [7]:
len(test.class_indices.keys())

In [8]:
classifier = classifier_resnet.output
classifier = Flatten()(classifier)
classifier = Dropout(0.2)(classifier)
classifier = Dense(units=38, activation='softmax')(classifier)

model = Model(inputs = classifier_resnet.input , outputs = classifier)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

In [11]:
resnet_early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", mode="min", patience=5, restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=5, min_lr=0.000002)

In [12]:
# Fit the model
history = model.fit(train, validation_data = val, epochs = 10, batch_size = 32, shuffle = True, verbose = 1, 
                    callbacks = [resnet_early_stopping,reduce_lr])

In [13]:
# Saving the current model of resnet
model.save('/kaggle/working/resnet50_model_final.h5')

In [ ]:
# from keras.models import load_model
# model = load_model('/kaggle/working/resnet50_model1.h5')

In [21]:
score = model.evaluate(test, verbose = 0)
print('Accuracy for test images:', round(score[1] * 100, 3), '%')

score = model.evaluate(val, verbose = 0)
print('Accuracy for validation images:', round(score[1] * 100, 3), '%')

In [24]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Accuracy curve for Resnet50')
plt.ylabel('Accuracy')
plt.xlabel('Number of epochs')
plt.legend(['train', 'validation'], loc='lower right')
plt.savefig('accuracy_resnet50.png')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss curve for Resnet50')
plt.ylabel('Loss')
plt.xlabel('Number of epochs')
plt.legend(['train', 'validation'], loc='upper right')
plt.savefig('loss_resnet50.png')
plt.show()

In [25]:
def plot_confusion_matrix(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    plt.figure(figsize = (18, 18))
    ax = plt.subplot()
    plt.imshow(cm, interpolation = 'nearest', cmap = plt.cm.Purples)
    plt.colorbar()
    plt.title("Confusion Matrix")
    tick_marks = np.arange(len(uniq_labels))
    plt.xticks(tick_marks, uniq_labels, rotation=45)
    plt.yticks(tick_marks, uniq_labels)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    ax.title.set_fontsize(20)
    ax.xaxis.label.set_fontsize(16)
    ax.yaxis.label.set_fontsize(16)
    limit = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment = "center",color = "white" if cm[i, j] > limit else "black")
    plt.savefig('confusionmatrix_resnet50.png')
    plt.show()

In [26]:
true_value = []
resnet_pred = []
    
uniq_labels = sorted((f for f in os.listdir(train_dir) if not f.startswith(".")))
for idx, label in enumerate(uniq_labels):
    for file in [f for f in os.listdir(test_dir + "/" + label) if not f.startswith(".")]:
        true_value.append(test.class_indices[label])
        filepath = test_dir + "/" + label + "/" + file
        image = cv2.resize(cv2.imread(filepath), (128, 128))
        image_normalized = image / 255
            
        #resnet50
        resnet_50_image_prediction = np.argmax(model.predict(np.array([image_normalized])))
        resnet_pred.append(resnet_50_image_prediction)

In [27]:
from sklearn.metrics import confusion_matrix, classification_report
import itertools

# Confusion matrix for test data
plot_confusion_matrix(true_value, resnet_pred)

In [28]:
classes = test.class_indices.keys()
print(classification_report(true_value, resnet_pred, target_names = list(classes)))

In [51]:
print("Input label to be predicted from one of these (1, 2, 3, 4, 5, 6, 7, 8, 9, A, B, BEST OF LUCK, C, D, E, F, G, H, I, I LOVE YOU, J, K, L, \
M, N, O, P, Q, R, S, SPACE, T, U, V, W, X, Y, Z)")
true_label = input().upper()

In [52]:
# For only one prediction
class_labels = {0: '1', 1:'2', 2:'3', 3:'4', 4:'5', 5: '6', 6:'7', 7:'8', 8:'9', 9: 'A', 10:'B', 11:'BEST OF LUCK', 12:'C', 13:'D', 14: 'E', 15:'F', 16:'G', 
                17:'H', 18:'I', 19:'I LOVE YOU', 20:'J', 21:'K', 22:'L', 23:'M', 24:'N', 25:'O', 26:'P', 27: 'Q', 28:'R', 29:'S', 30:'SPACE', 
                31:'T', 32:'U', 33:'V', 34:'W', 35: 'X', 36:'Y', 37:'Z'}
filepath = filepath = test_dir + "/" + true_label + "/" + true_label + "1001.jpg"
img = cv2.resize(cv2.imread(filepath), (128,128))
# img = cv2.resize(cv2.imread('../input/american-sign-language-dataset/dataset/test/' +true_label+ '/' + true_label + '1100.jpg'),(128,128))
img_normalized = img/255
resnet_50_image_prediction = np.argmax(model.predict(np.array([img_normalized])))
plt.title(f"Prediction Class = {class_labels[resnet_50_image_prediction]}\n True Class = {true_label}")
plt.imshow(img)

if resnet_50_image_prediction in class_labels.keys():
    prediction = class_labels[resnet_50_image_prediction]
else:
    prediction = "Not Found"